<a href="https://colab.research.google.com/github/akashwaf/finvasia01/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# run to install library temporarily 

!git clone https://github.com/Shoonya-Dev/ShoonyaApi-py.git

# this are the libraries mentioned in requirements.txt
!pip install requests
!pip install websocket_client
!pip install /content/ShoonyaApi-py/dist/NorenRestApiPy-0.0.20-py2.py3-none-any.whl
!pip install pyyaml


!pip install pyotp


!pip install talib-binary

fatal: destination path 'ShoonyaApi-py' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ShoonyaApi-py/dist/NorenRestApiPy-0.0.20-py2.py3-none-any.whl
NorenRestApiPy is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# manually installed imports 
import pyotp

In [3]:
# neccessary preinstalled imports 

import multiprocessing as mp 
import time
import math
import os , sys
import logging
import pandas as pd 
import pprint
import time
import yaml
import warnings 



In [4]:
feed_opened = False


simple_momentum_check = { }


In [5]:


if True:
  from NorenRestApiPy.NorenApi import  NorenApi
  from threading import Timer
  import pandas as pd
  import time
  import concurrent.futures

  api = None
  class Order:
      def __init__(self, buy_or_sell:str = None, product_type:str = None,
                  exchange: str = None, tradingsymbol:str =None, 
                  price_type: str = None, quantity: int = None, 
                  price: float = None,trigger_price:float = None, discloseqty: int = 0,
                  retention:str = 'DAY', remarks: str = "tag",
                  order_id:str = None):
          self.buy_or_sell=buy_or_sell
          self.product_type=product_type
          self.exchange=exchange
          self.tradingsymbol=tradingsymbol
          self.quantity=quantity
          self.discloseqty=discloseqty
          self.price_type=price_type
          self.price=price
          self.trigger_price=trigger_price
          self.retention=retention
          self.remarks=remarks
          self.order_id=None


      #print(ret)

      


  def get_time(time_string):
      data = time.strptime(time_string,'%d-%m-%Y %H:%M:%S')

      return time.mktime(data)


  class ShoonyaApiPy(NorenApi):
      def __init__(self):
          NorenApi.__init__(self, host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/', eodhost='https://shoonya.finvasia.com/chartApi/getdata/')
          global api
          api = self

      def place_basket(self, orders):

          resp_err = 0
          resp_ok  = 0
          result   = []
          with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

              future_to_url = {executor.submit(self.place_order, order): order for order in  orders}
              for future in concurrent.futures.as_completed(future_to_url):
                  url = future_to_url[future]
              try:
                  result.append(future.result())
              except Exception as exc:
                  print(exc)
                  resp_err = resp_err + 1
              else:
                  resp_ok = resp_ok + 1

          return result
                  
      def placeOrder(self,order: Order):
          ret = NorenApi.place_order(self, buy_or_sell=order.buy_or_sell, product_type=order.product_type,
                              exchange=order.exchange, tradingsymbol=order.tradingsymbol, 
                              quantity=order.quantity, discloseqty=order.discloseqty, price_type=order.price_type, 
                              price=order.price, trigger_price=order.trigger_price,
                              retention=order.retention, remarks=order.remarks)
          #print(ret)

          return ret





def do_login():
  #enable dbug to see request and responses
  logging.basicConfig(level=logging.DEBUG)

  #start of our program
  api = ShoonyaApiPy()

  #credentials
  user        = 'FA13620'
  pwd       =  '2012Ak@sh'
  token =     "52R5K462BA4H5TJ7SS746S5534364527"
  factor2     =  pyotp.TOTP(token).now()
  vc          = 'FA13620_U'
  app_key     = '9088c808ac3522b35705a4acd8a46d03'
  imei        = 'abc1234'




  ret = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=app_key, imei=imei)


  print(factor2)

  print(ret)



def get_token_df (opt_type= 'CE', expiry = '08-DEC-2022'):

  # read
      
  token_df = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip')
  


  banknifty_token  = token_df [token_df['Symbol'] == 'BANKNIFTY']
  


  banknifty_token  = banknifty_token [banknifty_token['Expiry'] == expiry]
  



  banknifty_token  = banknifty_token [banknifty_token['OptionType'] == opt_type]
  print(banknifty_token)


  banknifty_token.sort_values(by =  'StrikePrice', inplace = True)


  return banknifty_token




def make_ready_to_subscribe( token_df  ):

  print(token_df)

  token_list = token_df['Token'].tolist()

  
  # token_list = token_df[80:len(token_df)]['Token'].tolist()

  ready_to_subscribe = []
  for i in token_list:

    ready_to_subscribe.append( 'NFO|'+str(i) )


  print(ready_to_subscribe)


  return ready_to_subscribe







def select_opt_by_premium (premium  = 100):
  # returns token and price of the selected premium 
  # for that it looks into ce_lp_data which is updated by the websocket 

  diff  = 1000000
  nearest_price = 1000000
  nearest_token = 9000000000

  for x, y  in ce_lp_data.items():


    if abs(float(y) - premium) < diff :

      diff = abs(float(y) - premium)
      nearest_token = x
      nearest_price = float(y)


  
    # print(x , y )

  print('nearest token , nearest_price, and difference is ')
  print(nearest_token,nearest_price , diff)


  return nearest_token , nearest_price







if True: 
  # warnings.simplefilter("ignore")
  feed_opened = False
  feedJson = {}
  socket_opened = False
  orderJson = {}


  acknowledgement_data = {}
  ce_lp_data = {}
  pe_lp_data = {}




  def event_handler_feed_update(tick_data):
      
      global data_response_type
      global ce_lp_data


      # UPDATE LIVE LTP IN  GLOBAL JASON FORMAT VARIABLE 
      if 'lp' in tick_data:
        ce_lp_data[tick_data['tk']] =   tick_data['lp']


  def event_handler_order_update(order_data):
      print('orderupdate')
      print(order_data)

  def open_callback():
      global feed_opened
      feed_opened = True


  def close_callback():
      global feed_opened
      # feed_opened = False

  def setupWebSocket() :

    api.close_websocket()
    global fee_opened
    
    api.start_websocket( order_update_callback=event_handler_order_update,
                      subscribe_callback=event_handler_feed_update, 
                      socket_open_callback= open_callback,
                        socket_close_callback = close_callback)

    time.sleep(1)
    
    while(feed_opened==False):
        # print('here only')
        pass


    return True 




In [6]:
do_login()


333233
{'request_time': '10:40:58 02-12-2022', 'actid': 'FA13620', 'uname': 'AKASH RAJESHKUMAR PATEL', 'prarr': [{'prd': 'C', 's_prdt_ali': 'CNC', 'exch': ['NSE', 'BSE', 'NIPO']}, {'prd': 'M', 's_prdt_ali': 'NRML', 'exch': ['NFO', 'CDS', 'BCD', 'MCX']}, {'prd': 'I', 's_prdt_ali': 'MIS', 'exch': ['NSE', 'BSE', 'NFO', 'CDS', 'BCD', 'MCX']}, {'prd': 'H', 's_prdt_ali': 'CO', 'exch': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE']}, {'prd': 'B', 's_prdt_ali': 'BO', 'exch': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE']}], 'stat': 'Ok', 'susertoken': '128647c621085f9ea7dc4417f555556903ba3651922dbe44553c216ef278653c', 'email': 'AKSPMONEY1@GMAIL.COM', 'uid': 'FA13620', 'brnchid': 'HO', 'orarr': ['LMT', 'MKT', 'SL-LMT', 'SL-MKT'], 'exarr': ['NSE', 'NFO', 'CDS', 'MCX', 'BSE', 'BCD', 'NIPO'], 'values': ['1'], 'mws': {'1': []}, 'brkname': 'FINV', 'lastaccesstime': '1669957858'}


In [14]:
# token_df = get_token_df()

# tokens_ready = make_ready_to_subscribe(token_df)

# print(tokens_ready)

ce_tokens_df = get_token_df(opt_type = 'CE',  expiry = '08-DEC-2022')
ready_to_subscribe = make_ready_to_subscribe(token_df = ce_tokens_df)

# ready_to_subscribe = tokens_ready

print(ready_to_subscribe)




# it starts websocket 
global feed_opened 

if feed_opened == True :

  api.close_websocket()
  time.sleep(1)
  setupWebSocket()

api.subscribe(ready_to_subscribe, feed_type='t')


time.sleep(5)



token, price  = select_opt_by_premium (premium = 100)

selected_token = token 





      Exchange  Token  LotSize     Symbol           TradingSymbol  \
70867      NFO  57749       25  BANKNIFTY  BANKNIFTY08DEC22C53000   
70869      NFO  57747       25  BANKNIFTY  BANKNIFTY08DEC22C47100   
70871      NFO  57745       25  BANKNIFTY  BANKNIFTY08DEC22C46900   
70873      NFO  57743       25  BANKNIFTY  BANKNIFTY08DEC22C46800   
74954      NFO  52359       25  BANKNIFTY  BANKNIFTY08DEC22C52500   
...        ...    ...      ...        ...                     ...   
88104      NFO  36757       25  BANKNIFTY  BANKNIFTY08DEC22C46700   
88106      NFO  36755       25  BANKNIFTY  BANKNIFTY08DEC22C46600   
88465      NFO  36394       25  BANKNIFTY  BANKNIFTY08DEC22C47200   
89713      NFO  35120       25  BANKNIFTY  BANKNIFTY08DEC22C46100   
89716      NFO  35117       25  BANKNIFTY  BANKNIFTY08DEC22C45900   

            Expiry Instrument OptionType  StrikePrice  TickSize  Unnamed: 10  
70867  08-DEC-2022     OPTIDX         CE      53000.0      0.05          NaN  
70869  08-DEC

KeyboardInterrupt: ignored

In [10]:

# api.unsubscribe(ready_to_subscribe)

api.close_websocket()
feed_opened == False



True

In [11]:
# imp
# used for getting token df and token list 
# used for ready to subscribe tokens
ce_tokens_df = get_token_df(opt_type = 'CE',  expiry = '08-DEC-2022')
ready_to_subscribe = make_ready_to_subscribe(token_df = ce_tokens_df)



print(ready_to_subscribe)

ce_token_list = ce_tokens_df['Token'].tolist()
print(ce_token_list)

      Exchange  Token  LotSize     Symbol           TradingSymbol  \
70867      NFO  57749       25  BANKNIFTY  BANKNIFTY08DEC22C53000   
70869      NFO  57747       25  BANKNIFTY  BANKNIFTY08DEC22C47100   
70871      NFO  57745       25  BANKNIFTY  BANKNIFTY08DEC22C46900   
70873      NFO  57743       25  BANKNIFTY  BANKNIFTY08DEC22C46800   
74954      NFO  52359       25  BANKNIFTY  BANKNIFTY08DEC22C52500   
...        ...    ...      ...        ...                     ...   
88104      NFO  36757       25  BANKNIFTY  BANKNIFTY08DEC22C46700   
88106      NFO  36755       25  BANKNIFTY  BANKNIFTY08DEC22C46600   
88465      NFO  36394       25  BANKNIFTY  BANKNIFTY08DEC22C47200   
89713      NFO  35120       25  BANKNIFTY  BANKNIFTY08DEC22C46100   
89716      NFO  35117       25  BANKNIFTY  BANKNIFTY08DEC22C45900   

            Expiry Instrument OptionType  StrikePrice  TickSize  Unnamed: 10  
70867  08-DEC-2022     OPTIDX         CE      53000.0      0.05          NaN  
70869  08-DEC

In [12]:
print(type(ce_lp_data))
print(ce_lp_data)

print(len(ready_to_subscribe))
print(len(ce_lp_data))

<class 'dict'>
{}
126
0


In [ ]:



ltp = price 
token = token 
def set_simple_momentum(token , ltp):

  # set dictionary which will be checked continuously.
  global simple_momentum_check

  simple_momentum_check [token]  = {}

  simple_momentum_check[token]['level'] =  ltp - ltp *0.001 
  simple_momentum_check[token]['side'] =  'down'
  simple_momentum_check[token]['condition'] =  'False'
  simple_momentum_check[token]['action'] =  'sell'           
  simple_momentum_check[token]['action_successfull'] =  'NS'             # not successfull = NS , successfull = S
  simple_momentum_check[token]['close'] =  'N'                           # NO = N , YES = Y 

  return True 




binary = set_simple_momentum(token , ltp )
if binary == True :

  print('done')
print( simple_momentum_check)
  






def continuous_checking ():
  # continuously checks for  condtion satisfied or not  i.e. simple momentum condition 



  global simple_momentum_check 


  counter = 0
  token_key_list  = list(simple_momentum_check.keys())


  print(len(simple_momentum_check))
  print(len(token_key_list))


  while len(simple_momentum_check) != 0:

    time.sleep(1)
    print(counter)

    token = token_key_list[counter]


    if  (float(ce_lp_data[token] )) <  simple_momentum_check[token] ['level'] :
      
         simple_momentum_check[token] ['action']  = True 

         print('take_some_acctions ')
         ######### take some actions ###### takes some trades ######### and get info updated 




    if counter +1 >= len(simple_momentum_check):
      counter = 0
    else:
      counter = counter + 1 

    
    time.sleep(1)
    
    
continuous_checking()




In [ ]:


selected_token_price = {'60695' : 90 , '60710': 85}
momentum_check  = {'60695': '' }


token_conditions  = { '60695' : { 'price': 110,
                                 'condition' : False,
                                 'action': False,
                                 'actition_success': False
                                 }
                     }


def token_condtion_action (token_price_dict = selected_token_price , movement = 'down'):
  



  ce_lp_data['60695']

  for x, y  in ce_lp_data.items():





  



In [ ]:
import multiprocessing as mp 
import time 


results_a = []
results_b = []
results_c = []



def func1(numbers):
  for number in numbers:
    results_a.append(math.sqrt( number**3))



def func2(numbers):
  for number in numbers:
    results_b.append(math.sqrt( number**4))

    
def func3(numbers):
  for number in numbers:
    results_c.append(math.sqrt( number**5))



if __name__ == '__main__':
  
  numberlist = list(range(5000))

  print('multiprocessing started')


  p1 = mp.Process(target = func1  )
  p2 = mp.Process(target = func2 )
  p3 = mp.Process(target = func3 )

  
  start = time.time()
  p1.start()
  p2.start()
  p3.start()
  end = time.time()


  
  print(end  - start )




multiprocessing started


Process Process-4:
Traceback (most recent call last):


0.021900415420532227


Process Process-5:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process Process-6:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
TypeError: func1() missing 1 required positional argument: 'numbers'
TypeError: func3() missing 1 required positional argument: 'numbers'
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
TypeError: func2() missing 1 required positional argument: 'numbers'


In [ ]:
print(results_a[0:10])

[0.0, 1.0, 2.8284271247461903, 5.196152422706632, 8.0, 11.180339887498949, 14.696938456699069, 18.520259177452136, 22.627416997969522, 27.0]


In [ ]:
response = {'t': 'tk',
            'e': 'NFO',
            'tk': '60691',
            'ts': 'BANKNIFTY24NOV22C40300',
            'pp': '2',
            'ls': '25', 
            'ti': '0.05', 
            'lp': '2119.50', 
            'pc': '2.83',
            'ft': '1669111200', 
            'o': '2105.00',
            'h': '2155.00',
            'l': '2078.60', 
            'c': '2061.15', 
            'ap': '2126.66', 
            'v': '300', 
            'oi': '7450',
            'poi': '7500', 
            'bp1': '2203.75',
            'sp1': '2215.35', 
            'bq1': '75', 
            'sq1': '175'}
